In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [37]:
import numpy as np
from copy import deepcopy

dirs=[ (0,1), (-1,0), (0,-1), (1,0)]

class PathState:
    def __init__(self,loc,score, direction, came_from=None):
        self.loc = loc
        self.score = score
        self.direction = direction
        self.came_from = came_from

    def give_neighbors(self,score_map):
        neighs=[]
        nR,nC = len(score_map),len(score_map[0])
        y,x = self.loc
        ny,nx = self.direction

        if score_map[y+ny][x+nx] != '#':
            neighs.append(  PathState((y+ny,x+nx), 
                                self.score+1, 
                                self.direction,
                                self)
                         )
        idir = dirs.index(self.direction)
        for ix in [-1,1]: 
            new_dir = dirs[int((idir+ix)%4)]
            neighs.append(PathState((y,x), 
                                self.score+1000, 
                                new_dir,
                                self)
                         )
        
        return neighs
                   
    def __repr__(self):
        return f'({self.loc[0]},{self.loc[1]}) with {self.score} going {self.direction}'

    def __str__(self):
        locs=[str(i) for i in self.loc]
        d=[str(i) for i in self.direction]
        return f'{"-".join(locs)}_{"-".join(d)}'
    
    def __lt__(self,other):
        return self.score < other.score
    

In [38]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, PathState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: PathState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> PathState:
        return heapq.heappop(self.elements)[1]

In [39]:
from collections import deque

def perform_breadth_first(score_map, start):
     # print out what we find
    frontier = PriorityQueue()
    s= PathState( start, 0, (0,1) )
    frontier.put( s , priority=0)
    
    seen= {str(s):0}
    
    while not frontier.empty() :
        cur = frontier.get()
        
        y,x = cur.loc
        if score_map[y][x] == "E":
            return cur.score
        
        for n_loc in cur.give_neighbors(score_map):
            
            key= str(n_loc)
            if key not in seen.keys() or seen[key] > n_loc.score:
                seen[key]=n_loc.score
                frontier.put(n_loc, n_loc.score)

    return None

In [40]:
def part_one(path='input_data/test_16.txt'):
    
    with open(path,'r') as f:
        score_map=[]
        start=None
        for y,line in enumerate(f.readlines()):
            line=line.strip()
            if 'S' in line:
                x = line.index('S')
                start= (y,x)
            score_map.append(list(line))

    steps = perform_breadth_first(score_map, start)
    
    
    return steps

In [41]:
%%time
part_one()

CPU times: user 7.26 ms, sys: 920 µs, total: 8.18 ms
Wall time: 7.37 ms


11048

In [43]:
%%time
part_one('input_data/day_16.txt')

CPU times: user 179 ms, sys: 3.22 ms, total: 182 ms
Wall time: 186 ms


91464

# Part 2

In [61]:
from collections import deque

def perform_breadth_first_paths(score_map, start):
     # print out what we find
    frontier = PriorityQueue()
    s= PathState( start, 0, (0,1) )
    frontier.put( s , priority=0)
    
    seen= {str(s):0}
    end_steps=None
    end_paths=[]
    
    while not frontier.empty() :
        cur = frontier.get()
        
        y,x = cur.loc
        if score_map[y][x] == "E":
            if end_steps is None:
                end_steps = cur.score
            if end_steps == cur.score:
                end_paths.append(cur)
            else:
                return end_paths
        
        for n_loc in cur.give_neighbors(score_map):
            
            key= str(n_loc)
            if key not in seen.keys() or seen[key] >= n_loc.score:
                seen[key]=n_loc.score
                frontier.put(n_loc, n_loc.score)

    return None

In [62]:
def read_path(position):
    point=[]
    while position is not None:
        point.append(position.loc)
        position = position.came_from
        
    return set(point)

In [65]:
def part_two(path='input_data/test_16.txt'):
    
    with open(path,'r') as f:
        score_map=[]
        start=None
        for y,line in enumerate(f.readlines()):
            line=line.strip()
            if 'S' in line:
                x = line.index('S')
                start= (y,x)
            score_map.append(list(line))

    paths = perform_breadth_first_paths(score_map, start)
    premium=set([])
    for p in paths:
        points=read_path(p)
        premium = premium | points
    return len(premium)

In [66]:
%%time
part_two()

CPU times: user 8.69 ms, sys: 1.27 ms, total: 9.96 ms
Wall time: 8.93 ms


64

In [68]:
%%time
part_two('input_data/day_16.txt')

CPU times: user 1min 5s, sys: 108 ms, total: 1min 5s
Wall time: 1min 6s


494